Kaggle learning course review
===

---

### 1) Pandas

#### 1) 데이터 준비(wine_dataset)

In [6]:
import pandas as pd
import numpy as np

wine_df = pd.read_csv('./CSV_files/wine_data.csv', index_col = 0)

In [9]:
wine_df.head(5)

,country,description,designation,points,price,province,region_1,region_2,variety,winery
0,US,This tremendous 100% varietal wine hails from ...,Martha's Vineyard,96,235.0,California,Napa Valley,Napa,Cabernet Sauvignon,Heitz
1,Spain,"Ripe aromas of fig, blackberry and cassis are ...",Carodorum Selección Especial Reserva,96,110.0,Northern Spain,Toro,NaN,Tinta de Toro,Bodega Carmen Rodríguez
2,US,Mac Watson honors the memory of a wine once ma...,Special Selected Late Harvest,96,90.0,California,Knights Valley,Sonoma,Sauvignon Blanc,Macauley
3,US,"This spent 20 months in 30% new French oak, an...",Reserve,96,65.0,Oregon,Willamette Valley,Willamette Valley,Pinot Noir,Ponzi
4,France,"This is the top wine from La Bégude, named aft...",La Brûlade,95,66.0,Provence,Bandol,NaN,Provence red blend,Domaine de la Bégude


In [12]:
wine_df.columns

Index(['country', 'description', 'designation', 'points', 'price', 'province',
       'region_1', 'region_2', 'variety', 'winery'],
      dtype='object')

#### 2) map - iterable한 객체의 요소를 지정된 함수로 처리해준다.

In [15]:
n_trop = wine_df.description.map(lambda x : 'tropical' in x).sum()
n_fruity = wine_df.description.map(lambda x : 'fruity' in x).sum()
descriptor_counts = pd.Series([n_trop, n_fruity], index=['tropical', 'fruity'])
descriptor_counts

tropical    4135
fruity      8669
dtype: int64

#### 3) apply - Dataframe 타입의 객체에서 호출하며, 행 또는 열 또는 전체 셀에 원하는 연산(메소드 적용)을 지원한다.

In [31]:
def rating_Algorithm(x):
    if x.points >= 95 or x.country == 'Canada':
        return 3
    elif 85 <= x.points < 95:
        return 2
    else:
        return 1
    
star_ratings = wine_df.apply(rating_Algorithm, axis = 'columns')
star_ratings

0         3
1         3
2         3
3         3
4         3
         ..
150925    2
150926    2
150927    2
150928    2
150929    2
Length: 150930, dtype: int64

#### 4) Grouping - 가장 많이 나온 점수 그룹 상위 3개 출력
- groupby를 사용하면 기본으로 그룹 라벨이 index가 된다.
- index를 사용하고 싶은 않은 경우에는 as_index=False 를 설정하면 된다.

In [45]:
wine_df.groupby(['points'], as_index=False).mean().head()

,points,price
0,80,17.224236
1,81,17.665078
2,82,19.171879
3,83,18.013604
4,84,19.367885


In [46]:
wine_df.groupby(['points']).mean().head()

,price
points,
80,17.224236
81,17.665078
82,19.171879
83,18.013604
84,19.367885


In [25]:
wine_df.groupby('points').points.count().sort_values(ascending=False).head(3)

points
87    20747
88    17871
90    15973
Name: points, dtype: int64

#### 5) Grouping (Multi-indexes) - 여러 columns 를 그룹으로 만들고, 그룹 중 최고 points에 대한 Dataframe 반환

In [66]:
wine_df.groupby(['country', 'province']).apply(lambda df: df.loc[df.points.idxmax()]).head(5)

country  \
country   province                      
Albania   Mirditë             Albania   
Argentina Mendoza Province  Argentina   
          Other             Argentina   
Australia Australia Other   Australia   
          New South Wales   Australia   

                                                                  description  \
country   province                                                              
Albania   Mirditë           This garnet-colored wine made from 100% Kallme...   
Argentina Mendoza Province  If the color doesn't tell the full story, the ...   
          Other             Take note, this could be the best wine Colomé ...   
Australia Australia Other   This big wine presents a sophisticated bouquet...   
          New South Wales   This wine's deep brassy color suggests honey, ...   

                                   designation  points  price  \
country   province                                              
Albania   Mirditë                          NaN      88   20.0   
Argentina Mendoza Province    Nicasia Vineyard      97  120.0   
          Other                        Reserva      95   90.0   
Australia Australia Other             Yattarna      92   65.0   
          New South Wales   Noble One Botrytis      93   32.0   

                                    province                 region_1  \
country   province                                                      
Albania   Mirditë                    Mirditë                      NaN   
Argentina Mendoza Province  Mendoza Province                  Mendoza   
          Other                        Other                    Salta   
Australia Australia Other    Australia Other  South Eastern Australia   
          New South Wales    New South Wales          New South Wales   

                           region_2     variety                winery  
country   province                                                     
Albania   Mirditë               NaN     Kallmet                Arbëri  
Argentina Mendoza Province      NaN      Malbec  Bodega Catena Zapata  
          Other                 NaN      Malbec                Colomé  
Australia Australia Other       NaN  Chardonnay              Penfolds  
          New South Wales       NaN    Sémillon            De Bortoli

#### 6) get_group -  그룹 안의 데이터 확인

In [59]:
wine_df.groupby(['country', 'province']).get_group(('Australia', 'Australia Other')).head()

,country,description,designation,points,price,province,region_1,region_2,variety,winery
3822,Australia,"A terrific value in sparkling wine, this sligh...",T Series Chardonnay-Pinot Noir-Pinot Meunier,89,13.0,Australia Other,Australia,NaN,Sparkling Blend,Taltarni
6473,Australia,"This is a ripe, plump rosé that's highly “smas...",Breathing Space,88,NaN,Australia Other,Australia,NaN,Rosé,Evans & Tate
6663,Australia,"This is a ripe, plump rosé that's highly “smas...",Breathing Space,88,NaN,Australia Other,Australia,NaN,Rosé,Evans & Tate
8492,Australia,"Broad and ripe, this wine offers hints of lemo...",Reserve,87,12.0,Australia Other,South Eastern Australia,NaN,Chardonnay,Lismore Range
9136,Australia,You could do a lot worse at the U.S. Open than...,Classic,86,9.0,Australia Other,South Eastern Australia,NaN,Chardonnay,Jacob's Creek


#### 7) size - 각 그룹 사이즈 확인

In [69]:
wine_df.groupby(['country', 'province']).size()

country    province        
Albania    Mirditë                2
Argentina  Mendoza Province    4742
           Other                889
Australia  Australia Other      553
           New South Wales      246
                               ... 
Uruguay    Juanico               19
           Montevideo             3
           Progreso               5
           San Jose              15
           Uruguay               18
Length: 455, dtype: int64

#### 8) agg - 그룹별로 여러가지의 함수를 실행할 수 있게 한다. 

In [67]:
wine_df.groupby(['country']).price.agg([len, min, max]).head()

,len,min,max
country,,,
Albania,2,20.0,20.0
Argentina,5631,4.0,250.0
Australia,4957,5.0,850.0
Austria,3057,8.0,1100.0
Bosnia and Herzegovina,4,12.0,13.0


In [81]:
wine_df.groupby(['country']).price.agg([len, min, max]).reset_index().head()  # 인덱스 되돌리기

,country,len,min,max
0,Albania,2,20.0,20.0
1,Argentina,5631,4.0,250.0
2,Australia,4957,5.0,850.0
3,Austria,3057,8.0,1100.0
4,Bosnia and Herzegovina,4,12.0,13.0


#### 9) sort_values(), sort_index() - 정렬

In [87]:
wine_df.groupby(['country']).mean().sort_values(by = 'price', ascending = False).head()

,points,price
country,,
US-France,88.000000,50.000000
England,92.888889,47.500000
France,88.925870,45.619885
Hungary,87.329004,44.204348
Luxembourg,87.000000,40.666667


In [91]:
wine_df.groupby('price').points.max().sort_index()

price
4.0        86
5.0        90
6.0        90
7.0        90
8.0        91
         ... 
1300.0     96
1400.0    100
1900.0     98
2013.0     91
2300.0     99
Name: points, Length: 357, dtype: int64

In [94]:
sorted_varieties = wine_df.groupby('variety').price.agg(['min', 'max']).sort_values(by = ['min', 'max'], ascending = False)
sorted_varieties

,min,max
variety,,
Cabernet-Shiraz,150.0,150.0
Mazuelo,92.0,105.0
Carignan-Syrah,80.0,80.0
Syrah-Cabernet Franc,69.0,69.0
Nasco,65.0,65.0
...,...,...
Rabigato,NaN,NaN
Sacy,NaN,NaN
Sauvignon Blanc-Sauvignon Gris,NaN,NaN


#### 10) dtypes / astype - 데이터 타입 확인 / 타입 변경

In [104]:
wine_df.dtypes

country         object
description     object
designation     object
points           int64
price          float64
province        object
region_1        object
region_2        object
variety         object
winery          object
dtype: object

In [107]:
wine_df.points.astype('float64')

0         96.0
1         96.0
2         96.0
3         96.0
4         95.0
          ... 
150925    91.0
150926    91.0
150927    91.0
150928    90.0
150929    90.0
Name: points, Length: 150930, dtype: float64

#### 11) isnull / notnull - Missing Data를 찾아 boolean 형식으로 반환한다.

In [125]:
wine_df.loc[pd.isnull(wine_df.country)]

,country,description,designation,points,price,province,region_1,region_2,variety,winery
1133,NaN,Delicate white flowers and a spin of lemon pee...,Askitikos,90,17.0,NaN,NaN,NaN,Assyrtiko,Tsililis
1440,NaN,"A blend of 60% Syrah, 30% Cabernet Sauvignon a...",Shah,90,30.0,NaN,NaN,NaN,Red Blend,Büyülübağ
68226,NaN,"From first sniff to last, the nose never makes...",Piedra Feliz,81,15.0,NaN,NaN,NaN,Pinot Noir,Chilcas
113016,NaN,"From first sniff to last, the nose never makes...",Piedra Feliz,81,15.0,NaN,NaN,NaN,Pinot Noir,Chilcas
135696,NaN,"From first sniff to last, the nose never makes...",Piedra Feliz,81,15.0,NaN,NaN,NaN,Pinot Noir,Chilcas


#### 12) fillna - Missing Data의 NaN에 특정 데이터를 입력한다.

In [123]:
wine_df.province.fillna('Unknown')

0                 California
1             Northern Spain
2                 California
3                     Oregon
4                   Provence
                 ...        
150925        Southern Italy
150926             Champagne
150927        Southern Italy
150928             Champagne
150929    Northeastern Italy
Name: province, Length: 150930, dtype: object

#### 13) replace - 데이터를 A에서 B로 변경한다.

In [121]:
wine_df.designation.replace("Martha's Vineyard", 'Vineyard')

0                                     Vineyard
1         Carodorum Selección Especial Reserva
2                Special Selected Late Harvest
3                                      Reserve
4                                   La Brûlade
                          ...                 
150925                                     NaN
150926                          Cuvée Prestige
150927                           Terre di Dora
150928                         Grand Brut Rosé
150929                                     NaN
Name: designation, Length: 150930, dtype: object

#### 13) rename - 이름 변경

In [130]:
wine_df.rename(columns = {'points' : 'score'}, index = {0:'first_entry', 1: 'second_entry'}).head(5)

,country,description,designation,score,price,province,region_1,region_2,variety,winery
first_entry,US,This tremendous 100% varietal wine hails from ...,Vineyard,96,235.0,California,Napa Valley,Napa,Cabernet Sauvignon,Heitz
second_entry,Spain,"Ripe aromas of fig, blackberry and cassis are ...",Carodorum Selección Especial Reserva,96,110.0,Northern Spain,Toro,NaN,Tinta de Toro,Bodega Carmen Rodríguez
2,US,Mac Watson honors the memory of a wine once ma...,Special Selected Late Harvest,96,90.0,California,Knights Valley,Sonoma,Sauvignon Blanc,Macauley
3,US,"This spent 20 months in 30% new French oak, an...",Reserve,96,65.0,Oregon,Willamette Valley,Willamette Valley,Pinot Noir,Ponzi
4,France,"This is the top wine from La Bégude, named aft...",La Brûlade,95,66.0,Provence,Bandol,NaN,Provence red blend,Domaine de la Bégude


#### 14) rename_axis - 축 이름 변경

In [134]:
wine_df.rename_axis("wines", axis='rows').rename_axis("fields", axis='columns').head()

fields,country,description,designation,points,price,province,region_1,region_2,variety,winery
wines,,,,,,,,,,
0,US,This tremendous 100% varietal wine hails from ...,Vineyard,96,235.0,California,Napa Valley,Napa,Cabernet Sauvignon,Heitz
1,Spain,"Ripe aromas of fig, blackberry and cassis are ...",Carodorum Selección Especial Reserva,96,110.0,Northern Spain,Toro,NaN,Tinta de Toro,Bodega Carmen Rodríguez
2,US,Mac Watson honors the memory of a wine once ma...,Special Selected Late Harvest,96,90.0,California,Knights Valley,Sonoma,Sauvignon Blanc,Macauley
3,US,"This spent 20 months in 30% new French oak, an...",Reserve,96,65.0,Oregon,Willamette Valley,Willamette Valley,Pinot Noir,Ponzi
4,France,"This is the top wine from La Bégude, named aft...",La Brûlade,95,66.0,Provence,Bandol,NaN,Provence red blend,Domaine de la Bégude


## 2. Intermediate Machine Learning

#### 1) 데이터 준비

In [233]:
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

import pandas as pd
import numpy as np

X_full = pd.read_csv('./CSV_files/melb_data.csv/train.csv', index_col = 'Id')
X_test_full = pd.read_csv('./CSV_files/melb_data.csv/train.csv', index_col = 'Id')

X_full.dropna(axis = 0, subset = ['SalePrice'], inplace = True)  # 축 / 결측값 찾을 범위(부분집합) / 기존 DataFrame 변경 여부
y = X_full.SalePrice
X_full.drop(['SalePrice'], axis = 1, inplace = True)

X = X_full.select_dtypes(exclude = ['object'])  # 수치값만 사용
X_test = X_test_full.select_dtypes(exclude=['object'])

X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size = 0.8, test_size = 0.2, random_state = 0)

# Function for comparing different approaches
def score_dataset(X_train, X_valid, y_train, y_valid):
    model = RandomForestRegressor(n_estimators=100, random_state=0)
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return mean_absolute_error(y_valid, preds)

In [228]:
X_train.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
Id,,,,,,,,,,,,,,,,,,,,,
619,20,90.0,11694,9,5,2007,2007,452.0,48,0,...,774,0,108,0,0,260,0,0,7,2007
871,20,60.0,6600,5,5,1962,1962,0.0,0,0,...,308,0,0,0,0,0,0,0,8,2009
93,30,80.0,13360,5,7,1921,2006,0.0,713,0,...,432,0,0,44,0,0,0,0,8,2009
818,20,NaN,13265,8,5,2002,2002,148.0,1218,0,...,857,150,59,0,0,0,0,0,7,2008
303,20,118.0,13704,7,5,2001,2002,150.0,0,0,...,843,468,81,0,0,0,0,0,1,2006


In [229]:
print(X_train.shape)

missing_val_count_by_column = (X_train.isnull().sum())  # column 별 결측값 개수를 Series로 반환
print(missing_val_count_by_column[missing_val_count_by_column > 0])  # 결측값이 있는 row만 Series로 반환

(1168, 36)
LotFrontage    212
MasVnrArea       6
GarageYrBlt     58
dtype: int64


#### 2) Drop columns with missing values(결측값 제거)

In [239]:
cols_with_missing = [col for col in X_train.columns if X_train[col].isnull().any()]  # 결측값이 있는 columns가 담긴 리스트

reduced_X_train = X_train.drop(cols_with_missing, axis = 1)  # 결측값이 있는 열 제거
reduced_X_valid = X_valid.drop(cols_with_missing, axis = 1)

In [240]:
print("MAE (Drop columns with missing values):")
print(score_dataset(reduced_X_train, reduced_X_valid, y_train, y_valid))

MAE (Drop columns with missing values):
17837.82570776256


#### 3) Imputation with Missing Values (결측값 대체)

In [247]:
from sklearn.impute import SimpleImputer

# SimpleImputer -> default : 결측값을 평균값(mena)으로 대체
my_imputer = SimpleImputer()
imputed_X_train = pd.DataFrame(my_imputer.fit_transform(X_train))
imputed_X_valid = pd.DataFrame(my_imputer.transform(X_valid))

# Imputation 시, 열 이름이 사라지므로, 재설정 필요
imputed_X_train.columns = X_train.columns
imputed_X_valid.columns = X_valid.columns

In [246]:
print("MAE (Imputation):")
print(score_dataset(imputed_X_train, imputed_X_valid, y_train, y_valid))

MAE (Imputation):
18062.894611872147


In [253]:
final_imputer = SimpleImputer(strategy='median')  # 결측값을 중간값(median)으로 대체
final_X_train = pd.DataFrame(final_imputer.fit_transform(X_train))
final_X_valid = pd.DataFrame(final_imputer.transform(X_valid))

final_X_train.columns = X_train.columns
final_X_valid.columns = X_valid.columns

In [255]:
model = RandomForestRegressor(n_estimators=100, random_state=0)
model.fit(final_X_train, y_train)

preds_valid = model.predict(final_X_valid)
print("MAE (Your approach):")
print(mean_absolute_error(y_valid, preds_valid))

MAE (Your approach):
17791.59899543379
